In [99]:
# !pip uninstall py2neo
# !pip install py2neo==3.1.2
# !pip install neo4jupyter
import neo4jupyter
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [ ]:
1.处理数据

In [18]:
generate_data = open("./neo4j_data.json",'a+')
with open('./train_data.json','r') as train_data:
    for each_line in train_data:
        line = json.loads(each_line)
        for each_spo in line['spo_list']:
            each_spo['object_type'] = each_spo['object_type']['@value']
            each_spo['object'] = each_spo['object']['@value']
        generate_data.write(str(line))
        generate_data.write('\n')
generate_data.close()

2.然后把neo4j_data.json录入数据库

In [ ]:
from py2neo import Graph,Node,Relationship
import json
graph = Graph("bolt://localhost:7687/db/schemaKG/", username="neo4j", password="kuda")
graph.delete_all()
with open('./neo4j_data.json','r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            try:
                line = json.loads(line)
            except:
                continue
            for spo in line['spo_list']:
    #             创建节点
                subject_node = Node(spo['subject_type'],name=spo['subject'])
                object_node = Node(spo['object_type'],name=spo['object'])
                graph.create(subject_node)
                graph.create(object_node)
    #             创建关系
                relation = Relationship(subject_node,spo['predicate'],object_node)
                graph.create(relation)

3.生成id文件

In [1]:
from py2neo import Graph
from pandas import DataFrame

graph = Graph("bolt://localhost:7687/db/schemaKG/", username="neo4j", password="kuda")


with open("./pkubase_entity_id.txt","w") as f:
    data = DataFrame(graph.run("match (n) return id(n),n").data())
    for index,row in data.iterrows():
        f.write('<'+row['n']['name']+'>\t'+str(row['id(n)'])+'\n')

# store2neo4j.py

In [52]:
from py2neo import Graph,Node,Relationship
import json
graph = Graph("bolt://localhost:7687/db/schemaKG/", username="neo4j", password="kuda")
graph.delete_all()
with open('./neo4j_data.json','r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            try:
                line = json.loads(line)
            except:
                continue
            for spo in line['spo_list']:
    #             创建节点
                subject_node = Node(spo['subject_type'],name=spo['subject'])
                object_node = Node(spo['object_type'],name=spo['object'])
                graph.create(subject_node)
                graph.create(object_node)
    #             创建关系
                relation = Relationship(subject_node,spo['predicate'],object_node)
                graph.create(relation)

TransactionError: <py2neo.database.Transaction object at 0x7f8b6dbab748>

合并节点
MATCH (n)
WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count
WHERE count > 1
CALL apoc.refactor.mergeNodes(nodelist) YIELD node
RETURN node

删除重复关系
MATCH (a)-[r]->(b)
WITH a, b, TAIL (COLLECT (r)) as rr
FOREACH (r IN rr | DELETE r)

# 先尝试按原ganswer方法生成idmap

In [54]:
from py2neo import database
print(database.cursor.evaluate())

AttributeError: module 'py2neo.database' has no attribute 'cursor'

In [100]:
from py2neo import Graph
from pandas import DataFrame
graph = Graph("bolt://localhost:7687/db/schemaKG/", username="neo4j", password="kuda")
data = DataFrame(graph.run("match (n) where n.name='西门无恨' return n").data())
for index,row in data.iterrows():
    print(index)
    print(row['n'].labels())
    break

0


TypeError: 'LabelSetView' object is not callable

In [101]:
from py2neo import Graph
from pandas import DataFrame

graph = Graph("bolt://localhost:7687/db/schemaKG/", username="neo4j", password="kuda")

data = DataFrame(graph.run("match ()-[r]->() return distinct type(r)").data())
predicates = []
for index, row in data.iterrows():
   predicates.append(row.to_list()[0])
print(predicates)

['饰演', '主演', '成立日期', '上映时间', '母亲', '妻子', '父亲', '丈夫', '董事长', '毕业院校', '改编自', '代言人', '导演', '嘉宾', '歌手', '主持人', '作词', '作曲', '作者', '朝代', '国籍', '祖籍', '总部地点', '主题曲', '号', '出品公司', '配音', '主角', '所属专辑', '编剧', '校长', '所在城市', '首都', '简称', '制片人', '创始人', '获奖', '面积', '占地面积', '气候', '人口数量', '票房', '海拔', '注册资本', '官方语言', '修业年限', '邮政编码', '专业代码']


In [104]:
from py2neo import Graph
from pandas import DataFrame

data = DataFrame(graph.run("match (a) where a.name='天龙八部'  return id(a),labels(a)").data())
for row in data.iterrows():
    obj = row[1].to_dict()
    print(obj['labels(a)'])
    print(obj['id(a)'])

['图书作品', '影视作品']
2068


In [7]:
with open('/home/lx/PycharmProjects/kuda2.0/triple.txt', 'r') as f:
    i = 1
    for line in f:
        tri = line.split(' ')
        print(tri[2].strip())
        print(tri[0])
        print(tri[1])
        break

1
0
1


In [4]:
print(1)
print(2)

1
2
